In [29]:
import numpy as np
import cvxpy as cp
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl, wlexpr
print(cp.installed_solvers())

['CLARABEL', 'ECOS', 'ECOS_BB', 'OSQP', 'SCIPY', 'SCS']


In [21]:
sentiments = {
    'Haystacks': '+',
    'Ranch': '++',
    'Needles': '----',
    'SolarPanels': '---',
    'RedFlags': '++',
    'VR': '+++',
    'CosmicClean': '-',
    'Shirts': '+',
    'Coffee': '--'
}
returns = {
    '+': 0.02,
    '++': 0.15,
    '+++': 0.30,
    '-': -0.02,
    '--': -0.15,
    '---': -0.3,
    '----': -0.7
}

products = list(sentiments.keys())

In [36]:

rets = np.array([returns[sentiments[products[i-1]]] for i in range(1,10)])
pi = cp.Variable(9)
objective = cp.Minimize(120 * cp.sum_squares(pi) - 10000 * rets.T @ pi)
constraints = [cp.norm(pi, 1) <= 100]
prob = cp.Problem(objective, constraints)

prob.solve()
print('Optimal allocation without integer constraints:')
for i in range(9):
    print("Position in ", products[i], ': ', f"{pi.value[i]:,.2f}", '%', sep='')

Polishing not needed - no active set detected at optimal point
Optimal allocation without integer constraints:
Position in Haystacks: 0.83%
Position in Ranch: 6.25%
Position in Needles: -29.17%
Position in SolarPanels: -12.50%
Position in RedFlags: 6.25%
Position in VR: 12.50%
Position in CosmicClean: -0.83%
Position in Shirts: 0.83%
Position in Coffee: -6.25%


In [44]:
s1 = ' + '.join(['('+str(returns[sentiments[products[i-1]]])+')*p'+str(i)+'*10000-120*(p'+str(i)+')^2' for i in range(1,10)])
s2 = ' + '.join(['Abs[p'+str(i)+']' for i in range(1,10)]) + '<=100,'
s3 = ', '.join(['Element[p'+str(i)+', Integers]' for i in range(1,10)])
s4 = ', '.join(['p'+str(i) for i in range(1,10)])

In [45]:
with  WolframLanguageSession() as session:
    session.start()
    val_max, sol = session.evaluate(wlexpr('NMaximize[{'+s1+','+s2+s3+'}, {'+s4+'}]'))

Failed to converge to the requested accuracy or precision within 100 iterations.
Failed to converge to the requested accuracy or precision within 100 iterations.


In [46]:
print("Maximum profit achievable:", val_max)
print("Percentage of capital used: ", sum([abs(el[1]) for el in sol]), '%', sep='')

Maximum profit achievable: 153800.0
Percentage of capital used: 75%


In [47]:
for i in range(9):
    print("Position in ", products[i], ': ', sol[i][1], '%', sep='')

Position in Haystacks: 1%
Position in Ranch: 6%
Position in Needles: -29%
Position in SolarPanels: -12%
Position in RedFlags: 6%
Position in VR: 13%
Position in CosmicClean: -1%
Position in Shirts: 1%
Position in Coffee: -6%
